# Relevant Imports

In [31]:
showcase = True

In [32]:
import numpy as np
import pandas as pd
import glob
import os
import xgboost
import csv as csv
#import ipychart as ipc
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
# from sklearn.grid_search import GridSearchCV   #Perforing grid search
from scipy.stats import skew
from collections import OrderedDict

In [33]:
dataFrame = pd.read_csv("./CombinedData.csv", header = 0, low_memory=False)
dataFrame.insert(0, 'ID', range(0,len(dataFrame)))
#dataFrame = dataFrame.sample(frac=1).reset_index(drop=True)
print(dataFrame.head())

   ID         SOLD DATE PROPERTY TYPE                  ADDRESS        CITY  \
0   0      July-28-2022   Condo/Co-op      3881 Carl Parmer Dr  Harrisburg   
1   1    August-17-2022   Condo/Co-op        6419 Town Hall Pl  Harrisburg   
2   2  February-10-2022   Condo/Co-op  1 Buffalo Ave NW Unit 7     Concord   
3   3      March-4-2022   Condo/Co-op          476 Camrose Cir     Concord   
4   4    October-8-2021   Condo/Co-op      4068 Town Center Rd  Harrisburg   

  STATE OR PROVINCE ZIP OR POSTAL CODE   PRICE  BEDS  BATHS  \
0                NC              28075  258000   3.0    2.0   
1                NC              28075  242000   3.0    2.0   
2                NC              28025  102000   2.0    1.0   
3                NC              28025  203000   2.0    1.5   
4                NC              28075  170000   2.0    1.0   

                 LOCATION  SQUARE FEET  LOT SIZE  YEAR BUILT  $/SQUARE FEET  \
0              Harrisburg       1181.0       NaN      2005.0          218

In [34]:
dataFrame = dataFrame[dataFrame['SQUARE FEET'].notna()]
dataFrame = dataFrame[dataFrame['SOLD DATE'].notna()]
dataFrame = dataFrame[dataFrame['ZIP OR POSTAL CODE'].notna()]
dataFrame = dataFrame[dataFrame['YEAR BUILT'].notna()]
#dataFrame = dataFrame[dataFrame['LOCATION'].notna()]
dataFrame['SOLD DATE'] = pd.to_datetime(dataFrame['SOLD DATE']).apply(lambda x: x.value)

# Dropping Fields and Selecting Categorical Variables

In [35]:
dataFrame['ZIP OR POSTAL CODE'] = pd.Series([int(x.split('-')[0].split(' ')[0]) if '-' in x or ' ' in x else int(x) for x in dataFrame['ZIP OR POSTAL CODE']])

dataFrame = dataFrame.drop(columns=["URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)", 
                                   'CITY', 'STATE OR PROVINCE', 'ADDRESS', '$/SQUARE FEET'])
# dataFrame['ZIP OR POSTAL CODE'] = dataFrame['ZIP OR POSTAL CODE'].astype(int)
df = dataFrame.copy(deep=True)

In [36]:
h = dataFrame['PRICE'].quantile(0.99)
l = dataFrame['PRICE'].quantile(0.01)
dataFrame = dataFrame[(dataFrame["PRICE"] < h) & (dataFrame["PRICE"] > l)]

In [37]:
category_features = ['PROPERTY TYPE', 'LOCATION']
labels = dataFrame['PRICE']
training = dataFrame.drop(['ID'], axis = 1)

# Manipulating non-normalized dataframe

In [38]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [39]:
nan_features = ['LOCATION']
def ConvertToNAString(data, columnsList):
    for x in columnsList:
        data[x].apply(lambda x: x if x else 'NA')
ConvertToNAString(training, nan_features)

In [40]:
training.columns

Index(['SOLD DATE', 'PROPERTY TYPE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS',
       'BATHS', 'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT',
       'HOA/MONTH', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [41]:
training['LOCATION']

0                    Harrisburg
1        Harrisburg Town Center
2              Locke Mill Plaza
3                       Oakleaf
4        Harrisburg Town Center
                  ...          
78909             The Magnolias
78928                  Guilford
78929               Ridge Creek
78957                  Guilford
78961                High Point
Name: LOCATION, Length: 42942, dtype: object

In [42]:
non_categorical_columns = [col for col in training.columns if col not in category_features and col not in ['ID']]
numeric_features = training[non_categorical_columns].dtypes[training.dtypes != "object"].index

In [43]:
#training['HOA/MONTH'].head()

In [44]:
#training['HOA/MONTH'] = training['HOA/MONTH'].fillna(training['HOA/MONTH'].mean())

In [45]:
training[numeric_features] = np.log1p(training[numeric_features])

c:\users\yadav\appdata\local\programs\python\python39\lib\site-packages\pandas\core\internals\blocks.py:352: RuntimeWarning: invalid value encountered in log1p
  result = func(self.values, **kwargs)


In [46]:
#training[numeric_features].head()

In [47]:
training = pd.get_dummies(training, columns=category_features)

In [48]:
training.columns

Index(['SOLD DATE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'HOA/MONTH', 'LATITUDE',
       ...
       'LOCATION_vermillion', 'LOCATION_villages at Rosedale',
       'LOCATION_waters Edge', 'LOCATION_west asheville', 'LOCATION_wexford',
       'LOCATION_white oaks estates', 'LOCATION_windsor grove',
       'LOCATION_windstone Crossing', 'LOCATION_windy hill farms',
       'LOCATION_woodland hills'],
      dtype='object', length=6730)

In [49]:
X_train, X_test, y_train, y_test = train_test_split(training, labels, test_size=.3)
train_dataset = X_train
test_dataset = X_test
# train_dataset = pd.get_dummies(X_train,columns=category_features)
# test_dataset = pd.get_dummies(X_test,columns=category_features)

every_column_except_y= [col for col in train_dataset.columns if col not in ['PRICE','ID']]
train_X = train_dataset[every_column_except_y]
every_column_except_y= [col for col in test_dataset.columns if col not in ['PRICE','ID']]
test_X = test_dataset[every_column_except_y]
train_Y = train_dataset['PRICE']
test_Y = test_dataset['PRICE']

In [50]:
def createColumnPerValue(data, columnsList):
  for x in columnsList:
    values = pd.unique(data[x])

    for y in values: 
      column_name = x + "_" + str(y)
      data[column_name]=(data[x]==y).astype(float)
    
    data.drop(x, axis=1, inplace=True)



# Establish ML model

In [51]:
if not showcase:
    model = xgboost.XGBRegressor(colsample_bytree=0.4,
                     gamma=0,                 
                     learning_rate=0.07,
                     max_depth=3,
                     tree_method='gpu_hist', 
                     gpu_id=0,
                     min_child_weight=1.5,
                     n_estimators=10000,                                                                    
                     reg_alpha=0.75,
                     reg_lambda=0.45,
                     subsample=0.6,
                     seed=42)
else:
  model = xgboost.XGBRegressor()
  model.load_model("CurrentModel.json")

In [52]:
print(y_train)

38388    550000
62286    150000
71288    300000
62004    410000
20850    182000
          ...  
1339     226000
13262    392500
14183    205000
76289    217000
40831    314000
Name: PRICE, Length: 30059, dtype: int64


In [53]:
# every_column_except_y= [col for col in X_train.columns if col not in ['PRICE','ID']]
if not showcase:
    model.fit(train_X, train_Y)
OrderedDict(sorted(model.get_booster().get_fscore().items(), key=lambda t: t[1], reverse=True))

OrderedDict([('LATITUDE', 8297.0),
             ('LOT SIZE', 7686.0),
             ('YEAR BUILT', 7644.0),
             ('SQUARE FEET', 6900.0),
             ('SOLD DATE', 6178.0),
             ('ZIP OR POSTAL CODE', 5595.0),
             ('HOA/MONTH', 4110.0),
             ('BATHS', 2754.0),
             ('BEDS', 1820.0),
             ('LOCATION_Mooresville', 471.0),
             ('LOCATION_Asheville', 400.0),
             ('PROPERTY TYPE_Single Family Residential', 367.0),
             ('LOCATION_Durham', 339.0),
             ('LOCATION_Johnston', 329.0),
             ('PROPERTY TYPE_Multi-Family (2-4 Unit)', 258.0),
             ('LOCATION_Chatham', 243.0),
             ('PROPERTY TYPE_Condo/Co-op', 242.0),
             ('LOCATION_Gastonia', 202.0),
             ('LOCATION_Weaverville', 201.0),
             ('LOCATION_Statesville', 190.0),
             ('LOCATION_Alamance', 181.0),
             ('PROPERTY TYPE_Townhouse', 173.0),
             ('LOCATION_Salisbury', 172.0),
         

In [54]:
if not showcase:
    model.save_model("CurrentModel.json")

In [55]:
most_relevant_features= list( dict((k, v) for k, v in model.get_booster().get_fscore().items() if v >= 10).keys())

In [56]:
#y_pred = model.predict(test_X)
y_pred = np.expm1(model.predict(test_X))
predictions = [round(value) for value in y_pred]
predictions[:5]

[363023, 536465, 208402, 434188, 533809]

In [57]:
test_Y = np.expm1(test_Y)

In [58]:
test_Y.iloc[:5]

26311    380000.0
26206    661000.0
4532     120000.0
42918    565000.0
16712    365000.0
Name: PRICE, dtype: float64

In [59]:
errlist = []
ty = list(test_Y)
for i, v in enumerate(predictions):
    errlist.append(abs(ty[i]-predictions[i]))

## Mean Prediction Error in USD

In [60]:
f"${sum(errlist)/len(errlist):.2f}"

'$51868.28'